### Supervised learning algorithms:

supervised learning model:

- Use FY2015 icd9 diagnosis code long desciption as training data.
- Use "Other Diagnosis" column from Excel as test data

- Import Traning data, Separate input and output by slicing into 2 the arrays: one is input for training, the other is target. 
- Prepare y output target. Use sklearn.preprocessing.LabelEncoder to encode the Target which is ctr of icd9 dx code.
- Prepare X input array. Perform feature extraction by using klearn.feature_extraction.text  CountVectorizer with common "stop words" (ENGLISH_STOP_WORDS) against icd9 dx long desc.
- Create a corpus (bag-of-words with their usage statistics) from the terms that occur answers.
- Enhanced the input variable X with a weight for every term. Use klearn.feature_extraction.text import TfidfVectorizer
- Use class sklearn.svm.SVC as estimator to fit X/y to implements vector classification.

- Prepare X-test data ("Other Diagnosis") the same way as the training datasets, using TfidfVectorizer.
- Predict X_test with the model created above.





### Import Training data


In [1]:
import pandas as pd
import numpy as np
#filename = '~/Downloads/icd9dxref.fy15.txt'
url="https://raw.githubusercontent.com/azknowesis/team-collaboration/master/NLP/icd9dxref.fy15.txt"
df = pd.read_csv(url, header=None, delimiter='|', dtype=str, nrows=15000)
df.shape

# conver to array
df1=df.values

### Prepare X and y

In [2]:
# separate array into input and output components
inputx = df1[:,1:3]  #or X = df1, same result
outputy = df1[:,1]
#print(inputx) 
#print(outputy)

#Prepare Prepare Output variable y
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(outputy)

# check class list
#>>> list(le.classes_)

# transform y
y = le.transform(outputy)
#>>> y

## test le.transformer
#>>> list(le.inverse_transform([4, 2, 1]))

# Prepare Output variable X
# Option 1: use regular vectorizer with binary=True, default is True
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words

vectorizer = CountVectorizer(binary=True, stop_words=stop_words.ENGLISH_STOP_WORDS)
# vectorizer
corpus = inputx[:,1]
X = vectorizer.fit_transform(corpus)
X.toarray()


# # Option 2: use tfid weighted
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction import stop_words

# vectorizer_t = TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)

# corpus = inputx[:,1]
# X = vectorizer_t.fit_transform(corpus)
# X.toarray()

#>>> vectorizer_t.transform(['Something completely dystrophy.']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### Persist le encoder mapping for test datasets

In [4]:
# from joblib import dump, load
# dump(le, './joblib_dump/sklearn_MultinomialNB/le_15000_trnfit.joblib')

# from joblib import dump, load
# dump(vectorizer_t, './joblib_dump/sklearn_MultinomialNB/vectroizer_t_15000.pkl')

# from joblib import dump, load
# dump(vectorizer, './joblib_dump/vectroizer_binarytrue_15000.pkl')

['./joblib_dump/vectroizer_binarytrue_15000.pkl']

### Learning and predicting

In the case of the digits dataset, the task is to predict, given an image, which digit it represents. We are given samples of each of the 10 possible classes (the digits zero through nine) on which we fit an estimator to be able to predict the classes to which unseen samples belong.
In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T).
An example of an estimator is the class sklearn.svm.SVC, which implements support vector classification. The estimator’s constructor takes as arguments the model’s parameters.
For now, we will consider the estimator as a black box:

In [62]:
## probability=False by default. Turn off to increase processing speed.
#>>> from sklearn import svm
#>>> clf = svm.SVC(gamma=0.001, C=100.)
#>>> from sklearn.naive_bayes import GaussianNB
#>>> clf = GaussianNB()

In [6]:
from sklearn import svm

#>>> clf = svm.SVC(gamma=0.001, C=100. cache_size=200)
clf = svm.SVC(kernel="linear", C=0.025, cache_size=500)

# train the model
clf.fit(X, y)

#pred = clf.predict(vectors_test)
#metrics.f1_score(newsgroups_test.target, pred, average='macro')

SVC(C=0.025, cache_size=500, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [7]:
>>> from joblib import dump, load
# >>> dump(clf, './joblib_dump/sklearn_svm/clf_svclinear_15000_trnfit.joblib')
>>> dump(clf, './joblib_dump/sklearn_svm/clf_svclinear_binary_15000_trnfit.joblib')

['./joblib_dump/sklearn_svm/clf_svclinear_binary_15000_trnfit.joblib']

In [56]:
#Later, can load back the pickled model (possibly in another Python process) with:
#>>> clf = load('filename.joblib')

In [ ]:
# test score
>>> from sklearn import metrics

#>>> clf = MultinomialNB(alpha=.01)
#>>> clf.fit(X, y)

pred = clf.predict(X)
metrics.f1_score(y, pred, average='macro')

### Scores
clf = MultinomialNB(alpha=.01)
score: 0.9728996360448942

Now you can predict new values. In this case, you’ll predict using the last image from digits.data. By predicting, you’ll determine the image from the training set that best matches the last image.


In [23]:
X[:10]

<10x2763 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [8]:
#>>> clf.predict(X[:-1])
>>> clf.predict(X[:10])

array([0, 1, 2, 3, 6, 6, 6, 7, 8, 9])

##### How far is it from the boundary?
Function “decision_function” that computes the signed distance of a point from the boundary. A negative value would indicate class 0 and a positive value would indicate class 1. Also, a value close to 0 would indicate that the point is close to the boundary.


In [24]:
#>>> clf.decision_function(X[:10])

##### Class Probability
$P(class/input) = 1 / (1 + exp(A * f(input) + B))$

In [25]:
#>>> clf.predict_proba(X[:10])

In [58]:
#>>> list(le.inverse_transform(clf.predict(X[:-1])))
>>> list(le.inverse_transform(clf.predict(X[:10])))

['0010',
 '0011',
 '0019',
 '0020',
 '0021',
 '0021',
 '0021',
 '0029',
 '0030',
 '0031']

### Prepare Sample X_test

In [9]:
import pandas as pd
import numpy as np
#filename = '~/Downloads/icd9dxref.fy15.txt'
url="https://raw.githubusercontent.com/azknowesis/team-collaboration/master/NLP/oth_dx.txt"
df_test = pd.read_csv(url, header=None, delimiter='|', dtype=str, nrows = 100)
df_test.shape
#print(df)

(100, 2)

In [10]:
#df1 = df[df[1].str.contains("36")]
#df_test1=df_test[1]
df_test1 = df_test.iloc[0:100,1]
print(df_test1)
df_test1.shape

0                                 blind hypotensive eye
1                                              headache
2              concussion with no loss of consciousness
3                                   epiretinal membrane
4     subretinal and intraretinal hemorrhage od with...
5                          pdx traumatic vision loss od
6     suspect corneal abrasion due to irritation fro...
7                                        pseudo aphakia
8     s/p os 25c ppv/ilm el with icg/ivf/5f6 20% 1/8...
9                                              pciol od
10                 eye sensitivity to light photophobia
11    s/p os 256ppv/icm peel+fc6/ivf/sf6 20% macular...
12                                        eye trauma od
13                                    macular puckering
14                                     dry eye syndrome
15                 visit for military services physical
16                       s/p os 256 ppv/ic6/ivt/sf6 25%
17                                        eye tr

(100,)

Convert Dataframe to list

In [11]:
>>> inputx_test = df_test1.astype(str).values.tolist()
>>> inputx_test

['blind hypotensive eye',
 'headache',
 'concussion with no loss of consciousness',
 'epiretinal membrane',
 'subretinal and intraretinal hemorrhage od with choroidal rupture s/p recent globe exploration',
 'pdx traumatic vision loss od',
 'suspect corneal abrasion due to irritation from contact lense',
 'pseudo aphakia',
 's/p os 25c ppv/ilm el with icg/ivf/5f6 20% 1/8/09 2/2 post traumatic macular hole',
 'pciol od',
 'eye sensitivity to light photophobia',
 's/p os 256ppv/icm peel+fc6/ivf/sf6 20% macular hole closed',
 'eye trauma od',
 'macular puckering',
 'dry eye syndrome',
 'visit for military services physical',
 's/p os 256 ppv/ic6/ivt/sf6 25%',
 'eye trauma od',
 'strabismus non paralytic exotropia monocular od',
 'strabismus non paralytic hypertropia od',
 'patient education',
 'pdx chronic obstructive disease',
 'pciol od',
 'eye trauma',
 'foreign body intraocular unspecified',
 'abrasion of cornea',
 'pciol od',
 'ddx corneal abrasion',
 'ddx corneal ulcer',
 's/p ppv/il

In [12]:
# >>> X_test = vectorizer_t.transform(inputx_test)
>>> X_test = vectorizer.transform(inputx_test)
>>> X_test.toarray()
>>> X_test.shape

(100, 5913)

In [63]:
#>>> clf.predict(X_test)

In [13]:
>>> list(le.inverse_transform(clf.predict(X_test)))

['36041',
 '7840',
 '8500',
 '37133',
 '36363',
 '9585',
 '37182',
 '37931',
 '78033',
 'V620',
 'V411',
 '37022',
 'V411',
 '36256',
 '7580',
 'E0001',
 'V620',
 'V411',
 '37811',
 '37850',
 'V6549 B',
 '49320',
 'V620',
 'V411',
 '36060',
 'V595',
 'V620',
 '52125',
 '37006',
 '37161',
 '37161',
 'V790',
 '7014',
 '3671',
 '9409',
 '36107',
 '9585',
 '7014',
 'V411',
 '37542',
 'V411',
 '36181',
 '36229',
 '36130',
 '9181',
 '36031',
 '37133',
 '36011',
 '37931',
 '36616',
 'V411',
 '52412',
 '36256',
 '36471',
 '9181',
 'V411',
 '37239',
 'V4589',
 '7014',
 'V411',
 '36089',
 '37275',
 '3619',
 '3668',
 'V8279',
 '3619',
 '37931',
 'V411',
 '37022',
 '72631',
 '36105',
 '36181',
 '36229',
 '37133',
 '36031',
 '36011',
 '37925',
 '37931',
 '36616',
 'E0001',
 '8026',
 'V411',
 '37714',
 '36330',
 '25000',
 'V4589',
 '36089',
 'V411',
 '9514',
 '9514',
 '9514',
 '9181',
 '37530',
 '78659',
 '9213',
 '92231',
 '7014',
 '7014',
 '7092',
 '73819']